In [ ]:
import pandas as pd
import numpy as np

In [ ]:
actas_con_personas = pd.read_csv('../../../assets/silver/data_utilizada/personas_un_acta_por_fila.csv')
actas_con_personas['fecha'] = pd.to_datetime(actas_con_personas['fecha'], format='%Y-%m-%d')
actas_con_personas['dni'] = actas_con_personas['dni'].astype(str)
actas_con_personas.drop(columns = ['Unnamed: 0'], inplace = True)
actas_con_personas['dni'].nunique()

In [ ]:
actas_con_personas.columns

In [ ]:
actas = actas_con_personas[['dni', 'materia', 'carrera', 'tipo_acta', 'fecha', 'nota', 'resultado', 'semestre_relativo']].copy()

Siguiendo la misma logica que con el CBC, debemos tener las siguientes columnas:

Tenemos una fila por DNI y, como ahora definimos que queremos predecir qué pasa en el semestre relativo 4, tendríamos por cada semestre relativo j (j tomando valores 0 a 3 inclusive) las columnas:

* #inscripciones_j: como la que usamos para la target
* #TPs_aprobados_j: Para los TPS (excepto casos específicos) no tenemos nota numérica, solamente si aprobó, reprobó o dejó la materia, por eso tomamos la cantidad. En cambio para finales si tenemos el dato de la nota.
* #finales_inscriptos_j: como la que usamos para calcular la target

Hay que agregar las materias como columnas. En principio son 40 para la carrera.
La difrencia es que en el cbc es un cluster y en la carrera se ponen en el orden que haya rendido. 
Para cada materia:
hay que tomar fecha 0 como 1/1/2020 y a partir de ahi un numero natural que sean la cantidad de dias desde 1/1/2020. Luego, agregar la fecha de TP. agregar la nota tambien.  Esto es para materias aprobadas.

La idea es tener una distribucion empirica de que rinde, cada cuanto.

Entonces por cada una de las 9 materias en los semestres relativos 0 a 3 tenemos:
- 'fecha_TP_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'tp_aprobado': 1 si aprobó 0 si no?
- 'fecha_final_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'nota_final_materia_x'

Donde x va de 1 a 9 y si rindió la materia más de una vez, tenemos columnas distintas informando sobre las distintas veces que cursó.

Doy un ejemplo: Labo de datos TP aprobado el 1/2/2022 y final aprobado con 8 1/3/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 1
- 'fecha_final_materia_x' = 31+28 = 59
- 'nota_final_materia_x' = 8
Si no hubiera aprobado la materia, Labo de datos TP desaprobado/abandonado el 1/2/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 0
- 'fecha_final_materia_x' = dato faltante
- 'nota_final_materia_x' = 0

Luego, si la persona tiene más de 9 materias, completamos 4 columnas más:
- promedio de fecha
- promedio de nota de final
- 'tp_aprobado': pongo si aprobó algún TP

In [ ]:
actas['dni'].nunique()

## Primeras columnas

Para las primeras columnas, yo ya tenía un dataset que ayudará a calcular:

Tenemos una fila por DNI y, como ahora definimos que queremos predecir qué pasa en el semestre relativo 4, tendríamos por cada semestre relativo j (j tomando valores 0 a 3 inclusive) las columnas:

* #inscripciones_j: como la que usamos para la target
* #TPs_aprobados_j: Para los TPS (excepto casos específicos) no tenemos nota numérica, solamente si aprobó, reprobó o dejó la materia, por eso tomamos la cantidad. En cambio para finales si tenemos el dato de la nota.
* #finales_inscriptos_j: como la que usamos para calcular la target

In [ ]:
variables_resumen = pd.read_csv('../../../assets/silver/data_utilizada/resumen_personas_por_semestre.csv')

In [ ]:
variables_resumen['dni'] = variables_resumen['dni'].astype(str)

In [ ]:
variables_resumen.columns

In [ ]:
# Primero pivotamos el DataFrame
df_pivot = variables_resumen.pivot(index='dni', columns='semestre_relativo', 
                    values=['inscripciones', 'tp_aprobados', 'finales_inscriptos', 'total_actividad'])

# Aplanamos los nombres de columnas combinadas (MultiIndex)
df_pivot.columns = [f'{col}_{sem}' for col, sem in df_pivot.columns]

# Reiniciamos el índice para que 'dni' vuelva a ser una columna
df_pivot.reset_index(inplace=True)

In [ ]:
df_pivot.columns

In [ ]:
df_pivot[df_pivot['total_actividad_4.0']>=3].shape #igual que cuando calculé la target

Por ahora dejé las columnas del semestre relativo 4, pero para el entrenamiento no se pasarán

## Resto de columnas

Entonces por cada una de las 9 materias en los semestres relativos 0 a 3 tenemos:
- 'fecha_TP_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'tp_aprobado': 1 si aprobó 0 si no
- 'fecha_final_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'nota_final_materia_x'

Donde x va de 1 a 9 y si rindió la materia más de una vez, tenemos columnas distintas informando sobre las distintas veces que cursó.

Doy un ejemplo: Labo de datos TP aprobado el 1/2/2022 y final aprobado con 8 1/3/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 1
- 'fecha_final_materia_x' = 31+28 = 59
- 'nota_final_materia_x' = 8
Si no hubiera aprobado la materia, Labo de datos TP desaprobado/abandonado el 1/2/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 0
- 'fecha_final_materia_x' = dato faltante
- 'nota_final_materia_x' = 0

Luego, si la persona tiene más de 9 materias, completamos 4 columnas más:
- promedio de fecha
- promedio de nota de final
- 'tp_aprobado': pongo si aprobó algún TP

In [ ]:
actas.columns

Como primer paso, me tengo que quedar con las materias que sean anteriores al semestre relativo 4

In [ ]:
actas_entrenamiento = actas[actas['semestre_relativo'] != 4].copy()
actas_entrenamiento['semestre_relativo'].unique()

In [ ]:
actas_entrenamiento['dni'].nunique()

In [ ]:
actas_entrenamiento['nota'].unique()

In [ ]:
actas_entrenamiento[actas_entrenamiento['dni'] == '42950464']

Primero cambio la fecha por la fecha numerica

In [ ]:
# Suponiendo que ya tenés el DataFrame cargado como df
fecha_cero = pd.Timestamp("2020-01-01")
actas_entrenamiento['fecha_numerica'] = (actas_entrenamiento['fecha'] - fecha_cero).dt.days
actas_entrenamiento['fecha_numerica'].min(), actas_entrenamiento['fecha_numerica'].max()

In [ ]:
# Supongamos que actas_entrenamiento es tu DataFrame original
# Paso 1: Separar los TPS aprobados y desaprobados
is_tp = actas_entrenamiento['tipo_acta'] == 'Acta de Regulares/Promociones'
tp_aprobados = actas_entrenamiento[is_tp & (actas_entrenamiento['resultado'] == 'Aprobado')].copy()
tp_desaprobados = actas_entrenamiento[is_tp & (actas_entrenamiento['resultado'] != 'Aprobado')].copy()

# Paso 2: Separar los finales y hacer promedio si hay múltiples por persona y materia
finales = actas_entrenamiento[actas_entrenamiento['tipo_acta'] == 'Acta de Examen'].copy()
# reemplazo los Aprob. de la columna 'nota' por NaN para evitar problemas al promediar
finales['nota'] = finales['nota'].replace('Aprob.', np.nan).astype(float)

finales_grouped = finales.groupby(['dni', 'materia']).agg({
    'fecha_numerica': 'mean',
    'nota': 'mean'
}).reset_index()
finales_grouped.rename(columns={
    'fecha_numerica': 'fecha_final_materia',
    'nota': 'nota_final_materia'
}, inplace=True)

# Paso 3: Merge entre TP aprobados y finales
merged_tp_final = pd.merge(
    tp_aprobados[['dni', 'materia', 'fecha_numerica']],
    finales_grouped,
    on=['dni', 'materia'],
    how='outer',
    suffixes=('', '_final')
)
merged_tp_final.rename(columns={
    'fecha_numerica': 'fecha_TP_materia'
}, inplace=True)
merged_tp_final['tp_aprobado_materia'] = merged_tp_final['fecha_TP_materia'].apply(lambda x: 1 if pd.notna(x) else pd.NA)

# Paso 4: Agregar los TPs desaprobados
tp_desaprobados_agg = tp_desaprobados[['dni', 'materia', 'fecha_numerica']].copy()
tp_desaprobados_agg.rename(columns={
    'fecha_numerica': 'fecha_TP_materia'
}, inplace=True)
tp_desaprobados_agg['tp_aprobado_materia'] = 0
tp_desaprobados_agg['fecha_final_materia'] = pd.NA
tp_desaprobados_agg['nota_final_materia'] = pd.NA

# Unir con el anterior
full_actas_entrenamiento = pd.concat([merged_tp_final, tp_desaprobados_agg], ignore_index=True)

resultados = []
for dni, grupo in full_actas_entrenamiento.groupby('dni'):
    grupo = grupo.sort_values(by='fecha_TP_materia').reset_index(drop=True)
    row = {'Dni': dni}

    # Primeras 9 materias
    for i in range(min(9, len(grupo))):
        fila = grupo.iloc[i]
        idx = i + 1
        row[f'fecha_TP_materia_{idx}'] = fila['fecha_TP_materia']
        row[f'tp_aprobado_materia_{idx}'] = fila['tp_aprobado_materia']
        row[f'fecha_final_materia_{idx}'] = fila['fecha_final_materia']
        row[f'nota_final_materia_{idx}'] = fila['nota_final_materia']

    # Materias restantes para el resumen (10)
    if len(grupo) > 9:
        resto = grupo.iloc[9:]

        row['fecha_TP_materia_10'] = np.floor(resto['fecha_TP_materia'].mean(skipna=True))
        row['tp_aprobado_materia_10'] = 1 if (resto['tp_aprobado_materia'] == 1).any() else 0
        row['fecha_final_materia_10'] = np.floor(resto['fecha_final_materia'].mean(skipna=True))
        row['nota_final_materia_10'] = np.floor(resto['nota_final_materia'].mean(skipna=True))
    else:
        row['fecha_TP_materia_10'] = pd.NA
        row['tp_aprobado_materia_10'] = pd.NA
        row['fecha_final_materia_10'] = pd.NA
        row['nota_final_materia_10'] = pd.NA

    resultados.append(row)

final_df = pd.DataFrame(resultados)
# Ahora final_actas_entrenamiento tiene la estructura deseada

In [ ]:
final_df

In [ ]:
final_df.to_csv('../../../assets/silver/data_utilizada/actas_entrenamiento.csv', index=False)

In [ ]:
cuento = actas_entrenamiento[actas_entrenamiento['tipo_acta'] == 'Acta de Regulares/Promociones'].groupby(['dni', 'materia', 'resultado']).size().reset_index(name='count')

In [ ]:
cuento[(cuento['count'] > 1) & (cuento['resultado'] == 'Aprobado')]

In [ ]:
actas_entrenamiento[(actas_entrenamiento['dni'] == '96151169') & (actas_entrenamiento['materia'] == 'Análisis II')]

In [ ]:
carreras_promo = actas_entrenamiento[
    (actas_entrenamiento["tipo_acta"] == "Acta de Regulares/Promociones") &
    (actas_entrenamiento["resultado"] == "Aprobado")
][["dni", "materia", "carrera"]].drop_duplicates()

# 2. Hacemos merge para traer esa carrera a todas las filas del mismo dni y materia
df_merged = actas_entrenamiento.merge(
    carreras_promo,
    on=["dni", "materia"],
    how="left",
    suffixes=("", "_promo")
)

# 3. Filtramos solo las filas de Acta de Examen
examenes = df_merged[df_merged["tipo_acta"] == "Acta de Examen"]

# 4. Verificamos si hay alguna diferencia de carrera respecto a la promoción
condicion_incorrecta = examenes["carrera"] != examenes["carrera_promo"]
errores = examenes[condicion_incorrecta & examenes["carrera_promo"].notna()]

# 5. Mostramos los casos donde no se cumple la condición
print(errores)

In [ ]:
errores

In [ ]:
final_df['Dni'].nunique()